In [294]:
import generator
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Activation,LSTM,Bidirectional,Embedding,Conv2D,BatchNormalization,Activation,Masking,LeakyReLU
from tensorflow.keras.activations import tanh,relu
from tensorflow.keras.models import Model
from tensorflow.keras import Model as models
import numpy as np
import librosa
import matplotlib.pyplot as plt
import logging, os
from sklearn.metrics.pairwise import cosine_similarity

logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from IPython.display import display
from time import sleep

print('Test 1')
dh = display('Test2',display_id=True)
sleep(1)
dh.update('Test3')

files , json,json_word = generator.gen()
maxi = 1021

Test 1


'Test3'

In [196]:
#공식 구해야함
'''
maxi = 0
for k,i in enumerate(files):
    wav,_ = librosa.load(files[i]['file_path'],sr=16000)
    mfcc = librosa.feature.mfcc(wav,n_mfcc=40)
    if mfcc.shape[1] > maxi:
        maxi = mfcc.shape[1]
    if k%200==0:
        print(k)
'''


emb_layer = Embedding(len(json),512)
opt = tf.keras.optimizers.Adam(10e-3)
maxi = 1021
sos_emb = emb_layer(1)

emb_results = [None]*len(files)
mfccs_temp = []
#전체 파일 생성
for k,i in enumerate(files):
    if k % 20 == 0:
        print(k)
    file_word_num = tf.cast([ json[i][0] for i in files[i]['file_txt']],tf.float32)
    wav,_ = librosa.load(files[i]['file_path'],sr=16000)
    emb_result = emb_layer(file_word_num)
    emb_results[k] = emb_result
    mfcc = np.expand_dims(
                np.expand_dims(
                    librosa.feature.mfcc(wav,n_mfcc=40),axis=-1),axis=0)

    tak = np.zeros((1,40,maxi-mfcc.shape[-2],1),dtype=np.float32)
    mfcc = np.concatenate([mfcc,tak],axis=2)
    mfccs_temp.append(mfcc)
mfccs = tf.concat(mfccs_temp,axis=0)
    
max_len_txt = 0
for i in emb_results:
    if max_len_txt < i.shape[0]:
        max_len_txt = i.shape[0]

embs = []
for k,i in enumerate(emb_results):
    if k % 20 == 0:
        print(k)
    emb = np.zeros((max_len_txt-(i.shape[0]),512),dtype=np.float32)
    padded_emb = np.expand_dims(np.concatenate([i,emb],axis=0),axis=0)
    embs.append(padded_emb)
embs = np.concatenate(embs,axis=0)

train_mfccs = mfccs[:-270]    
test_mfccs = mfccs[-270:]

train_embs = embs[:-270]    
test_embs = embs[-270:]


0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
17

In [395]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects


def hard_tanh(x):
    return tf.minimum(20.0, tf.maximum(0.0, x)) 
    
get_custom_objects().update({'hard_tanh':Activation(hard_tanh)})
    
class encoder(Model):
    def conv_init(self):
        conv = Sequential([
            Conv2D(32,(21,81),strides=(2,2),padding='valid',input_shape=(40,1021,1)),
            BatchNormalization(-1),
            Activation(hard_tanh),
            Conv2D(32,(10,41),strides=(1,2),padding='valid'),
        ])
        
        return conv
    
    def __init__(self):
        super(encoder,self).__init__()
        self.conv = self.conv_init()
        self.rnn = Sequential([
            Masking(mask_value=0.0),
            LSTM(256,input_shape=(216,30)),
        ])
        
    def call(self,audio):
        x = self.conv(audio)
        x = self.rnn(tf.squeeze(x,1))
        
        return x
        
class seq2(Model):
    def __init__(self, enc, dec):
        super(seq2,self).__init__()
        self.enc = enc
        self.dec = dec
        
    def call(self,audio, txt_emb):
        enc_out = self.enc(audio)
        dec_out = self.dec(enc_out,txt_emb)
        
        return dec_out
    
    def predict(self, audio, sos_emb):
        enc_out = self.enc(audio)
        tiled = tf.tile(tf.expand_dims(enc_out,axis=1),[1,94,1])
        
        
        sos_emb = tf.expand_dims(tf.expand_dims(sos_emb,axis=0),axis=1)
        
        predict_emb = tf.convert_to_tensor(np.zeros((1,93,512)),tf.float32)
        predict_emb = tf.concat([sos_emb,predict_emb],axis=1)
        predict_emb = tf.concat([predict_emb,tiled],axis=2)
        
        output, h , c = self.dec.rnn(predict_emb)
        hidden = [h,c]
        stop_sign = False
        while not stop_sign:
            output , h ,c = self.dec.rnn(
                tf.concat([output,tiled],axis=2),
                initial_state = hidden
            )
            hidden = [h,c]
            
            cur_word = output[0,-1,:]
            if 
        
        
        return output
        
        

class decoder(Model):
    def __init__(self):
        super(decoder,self).__init__()
        '''self.rnn = Sequential([
            Masking(mask_value=0.0),
            LSTM(512,input_shape=(94,768),return_sequences=True,return_state=True)
        ])'''
        self.mask = Masking(mask_value=0.0)
        self.rnn = LSTM(512,input_shape=(94,768),return_sequences=True,return_state=True)
        '''
        def lstm_model():
            inp = tf.keras.Input(shape=(94,768))
            out = LSTM(512,return_sequences=True, return_state=True)(inp)
            return models(inputs=inp,outputs=out)
        self.rnn = lstm_model()'''
    
    def call(self,enc_out,embs):
        
        tiled = tf.tile(tf.expand_dims(enc_out,axis=1),[1,embs.shape[1],1])
        cat = tf.concat([tiled,embs],axis=2)
        cat = self.mask(cat)
        result,_,_ = self.rnn(cat)
        
        return result


In [392]:
'''
            print('loss : ',loss)
            print('embs : ',batch_emb[0,0,:10],batch_emb.shape)
            print('result : ',result[0,0,:10],result.shape)
            print('grad : ',grad[0][0][0,0,:10])
            print('weights : ',model.trainable_weights[0][0][0,0,:10])
            '''

'''
            print('after weights : ',model.trainable_weights[0][0][0,0,:10])
            print('\n\n')
            '''

"\n            print('after weights : ',model.trainable_weights[0][0][0,0,:10])\n            print('\n\n')\n            "

In [396]:
enc = encoder()
dec = decoder()
model = seq2(enc,dec)
opt = tf.keras.optimizers.Adam(10e-3)

#train

train_loss = tf.keras.metrics.Mean(name='train_loss')
df = display('start',display_id=True)

with tf.device(f'GPU:{0}'):
    for iter_num in range(100):
        for i in range(train_embs.shape[0]//30):
            batch_emb = train_embs[i*30:(i+1)*30]
            batch_mfcc = train_mfccs[i*30:(i+1)*30]
            with tf.GradientTape() as tape:
                
                result = model(batch_mfcc,batch_emb)
                loss=tf.reduce_mean(tf.reduce_mean(tf.math.sqrt(tf.reduce_sum(tf.math.pow(batch_emb-result,2),2)),1))
                train_loss(loss)
            grad = tape.gradient(loss,model.trainable_weights)
            opt.apply_gradients(zip(grad,model.trainable_weights))
            
        print(train_loss.result().numpy())
        df.update(tf.math.sqrt(tf.reduce_sum(tf.math.pow(batch_emb-result,2),2))[0])

<tf.Tensor: shape=(94,), dtype=float32, numpy=
array([0.01024733, 0.29427275, 0.5666514 , 0.23815371, 0.6211317 ,
       0.61200434, 0.52551174, 0.6478926 , 0.60902214, 0.5437035 ,
       0.6052846 , 0.43630534, 0.57811135, 0.40631077, 0.6312796 ,
       0.16245447, 0.64181   , 0.19111931, 0.66025984, 0.62025344,
       0.51852834, 0.33884102, 0.627501  , 0.671219  , 0.44756696,
       0.47678572, 0.5334722 , 0.2189372 , 0.52149075, 0.63598233,
       0.43174124, 0.47607934, 0.49581918, 0.62269574, 0.45155302,
       0.6493241 , 0.5064363 , 0.5332242 , 0.5451345 , 0.12990975,
       0.0843893 , 0.05803268, 0.04183856, 0.03121864, 0.02403545,
       0.01905975, 0.015591  , 0.01317345, 0.01148528, 0.01035007,
       0.00960079, 0.00911697, 0.00880526, 0.00860653, 0.00848025,
       0.00839974, 0.00834818, 0.00831453, 0.00829207, 0.0082765 ,
       0.00826529, 0.00825692, 0.00825047, 0.00824533, 0.00824113,
       0.00823765, 0.00823473, 0.00823226, 0.00823019, 0.00822844,
       0.008226

0.23333797


KeyboardInterrupt: 

In [397]:
model.predict(tf.expand_dims(test_mfccs[3],axis=0),sos_emb)

(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)

(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)

(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)

(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)

(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)
(1, 94, 512)

KeyboardInterrupt: 

In [387]:
model.dec.summary()

Model: "decoder_99"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_209 (LSTM)              multiple                  2623488   
Total params: 2,623,488
Trainable params: 2,623,488
Non-trainable params: 0
_________________________________________________________________


In [362]:
enc.rnn(tf.squeeze(enc.conv(tf.expand_dims(test_mfccs[3],axis=0)),0))

<tf.Tensor: shape=(1, 256), dtype=float32, numpy=
array([[-0.01582424, -0.00425652,  0.008129  ,  0.0099589 , -0.00204774,
        -0.00432909, -0.00154349,  0.00631252,  0.00590705,  0.00289897,
        -0.0008629 ,  0.00890549,  0.01140664,  0.01150629, -0.00276578,
         0.03366494, -0.00443113, -0.00541268, -0.0101196 , -0.00032859,
         0.0025227 , -0.0308333 , -0.01246279,  0.00648391, -0.02159143,
         0.00675611,  0.00017525, -0.00290504, -0.00103804, -0.01047956,
         0.01053063,  0.0355453 ,  0.00172385,  0.0089662 ,  0.02359475,
         0.01683112,  0.0083011 , -0.00592119,  0.00205823, -0.01358383,
        -0.00138489,  0.00756344,  0.00671917,  0.00690426,  0.00505948,
        -0.00685399, -0.00561809, -0.00073903, -0.01204374, -0.0133612 ,
         0.00545522,  0.01383715,  0.00931941, -0.00229982, -0.00076074,
        -0.01286763, -0.00121031,  0.01367242, -0.06696991, -0.00692266,
         0.00188371,  0.00398878, -0.00290183, -0.00243847, -0.00011299,
 

In [250]:
enc = encoder()
dec = decoder()
model = seq2(enc,dec)
test = enc.rnn(tf.squeeze(enc.conv(batch_mfcc)))
tere = cosine_similarity(tf.expand_dims(test[15],axis=0),tf.expand_dims(test[20],axis=0))

In [251]:
dec(test,batch_emb)

AttributeError: 'list' object has no attribute '_keras_mask'

In [436]:
new_b = tf.squeeze(tf.transpose(batch_mfcc,[0,2,1,3]))
print(new_b.shape)
new_rnn = LSTM(512,input_shape=(40,1),return_sequences=True, return_state=True)
out,h,c = new_rnn(new_b)

(30, 1021, 40)


In [437]:
out

<tf.Tensor: shape=(30, 1021, 512), dtype=float32, numpy=
array([[[-3.57713566e-06, -6.04463182e-08,  7.04952180e-01, ...,
         -1.59827003e-04, -8.81839136e-04,  4.60493757e-05],
        [-1.20351942e-05, -2.05225689e-07,  8.74275565e-01, ...,
         -2.52397382e-04, -1.47916318e-03,  8.69278956e-05],
        [-8.58573640e-06, -9.32473313e-07,  9.55563247e-01, ...,
         -3.46110668e-04, -1.73912442e-03,  8.49464413e-05],
        ...,
        [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00]],

       [[-5.91452372e-05, -5.90770348e-08,  4.40216780e-01, ...,
         -2.10426151e-05, -2.88888868e-02,  2.90521493e-05],
        [-9.36568467e-05, -1.81631435e-07,  6.

In [430]:
new_mf = tf.expand_dims(librosa.feature.mfcc(wav,n_mfcc=40),axis=0)
new_mf = tf.transpose(new_mf,[0,2,1])

In [431]:
new_mf.shape

TensorShape([1, 126, 40])

In [432]:
out,h,c = new_rnn(new_mf)

In [435]:
out

<tf.Tensor: shape=(1, 126, 512), dtype=float32, numpy=
array([[[-7.6159418e-01,  7.0021078e-06, -5.9706884e-09, ...,
          3.8012734e-03, -6.8495625e-01,  3.2842821e-05],
        [-9.6402758e-01,  1.2522150e-05, -1.3327309e-08, ...,
          3.0040005e-03, -6.7643702e-01,  2.9882267e-05],
        [-9.9505478e-01,  9.9690624e-06, -2.3976096e-08, ...,
          4.2823413e-03, -6.7204553e-01,  3.1998534e-05],
        ...,
        [-1.0000000e+00,  3.3958149e-05, -5.4996994e-09, ...,
          4.6896366e-03, -5.0479162e-01,  1.7514423e-05],
        [-1.0000000e+00,  3.0246658e-05, -6.7683268e-09, ...,
          3.8487089e-03, -6.4183462e-01,  1.1801569e-04],
        [-1.0000000e+00,  2.6515945e-05, -3.5233692e-09, ...,
          2.6690108e-03, -5.7686412e-01,  4.0658898e-04]]], dtype=float32)>